In [1]:
import torch
from sympy import shape
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from torchinfo import summary
from torchvision import datasets, transforms, models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


## Implementatation

In [2]:
class SEBlock(nn.Module):
    def __init__(self, channels, squeeze_channels):
        super().__init__()
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.linear1 = nn.Linear(channels, squeeze_channels)
        self.relu = nn.SiLU()
        self.linear2 = nn.Linear(squeeze_channels, channels)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        se = self.gap(x)
        se = se.flatten(start_dim=1)
        se = self.linear1(se)
        se = self.relu(se)
        se = self.linear2(se)
        se = self.sigmoid(se)
        se = se[..., None, None]
        x = x * se
        return x


class GroupedConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, n_groups, stride):
        super().__init__()
        self.group_channels = in_channels // n_groups
        self.grouped_conv_paths = nn.ModuleList([
            nn.Conv2d(self.group_channels, out_channels // n_groups, kernel_size=3, stride=stride, padding=1,
                      bias=False) for _ in range(n_groups)
        ])
    
    def forward(self, x):
        x = torch.concat([
            path(x[:, self.group_channels * i:self.group_channels * (i + 1), ...]) for i, path in
            enumerate(self.grouped_conv_paths)], dim=1)
        return x


class DepthwiseSeparableConvolution(nn.Module):
    def __init__(self, in_channels, squeeze_channels, out_channels, kernel_size, stride):
        super().__init__()
        self.depthwise_conv = nn.Sequential(
            # GroupedConvolution(in_channels, in_channels, in_channels, stride),
            nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride, padding=(kernel_size - 1) // 2, bias=False,
                      groups=in_channels),
            nn.BatchNorm2d(in_channels),
            nn.SiLU(),
        )
        self.se_block = SEBlock(in_channels, squeeze_channels)
        self.seeparable_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )
    
    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.se_block(x)
        x = self.seeparable_conv(x)
        return x

In [3]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expansion_factor, kernel_size, stride=1):
        super().__init__()
        squeeze_channels = in_channels//4
        self.expansion_factor = expansion_factor
        self.use_skip_connection = (stride == 1 and in_channels == out_channels)
        layers = [nn.Sequential(
            nn.Conv2d(in_channels, in_channels * expansion_factor, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(in_channels * expansion_factor),
            nn.SiLU(),
        )] if self.expansion_factor != 1 else []
        self.bottleneck = nn.Sequential(
            *layers,
            DepthwiseSeparableConvolution(in_channels * expansion_factor, squeeze_channels, out_channels, kernel_size, stride)
        )
    
    def forward(self, x):
        bottleneck = self.bottleneck(x)
        bottleneck = bottleneck + x if self.use_skip_connection else bottleneck
        return bottleneck


x_batch = torch.randn(32, 16, 112, 112)
print(InvertedResidualBlock(16, 24, 3, 3, 1)(x_batch).shape)

torch.Size([32, 24, 112, 112])


In [4]:
class EfficientNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(n_channels, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.SiLU()
        )
        self.conv2 = InvertedResidualBlock(32, 16, 1, 3, 1)
        self.conv3 = self.make_layers(16, 24, 6, 2, 3, 2)
        self.conv4 = self.make_layers(24, 40, 6, 2, 5, 2)
        self.conv5 = self.make_layers(40, 80, 6, 3, 3, 2)
        self.conv6 = self.make_layers(80, 112, 6, 3, 5, 1)
        self.conv7 = self.make_layers(112, 192, 6, 4, 5, 2)
        self.conv8 = self.make_layers(192, 320, 6, 1, 3, 1)
        self.conv9 = nn.Sequential(
            nn.Conv2d(320, 1280, kernel_size=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.SiLU()
        )
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(1280, n_classes),
        )
    
    def make_layers(self, in_channels, out_channels, expansion_factor, n_blocks, kernel_size, stride):
        layers = [InvertedResidualBlock(in_channels, out_channels, expansion_factor, kernel_size, stride)]
        for layer in range(n_blocks - 1):
            layers.append(InvertedResidualBlock(out_channels, out_channels, expansion_factor, kernel_size))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x


x_batch = torch.randn(32, 3, 224, 224)
print(EfficientNet(3, 1000)(x_batch).shape)

torch.Size([32, 1000])


In [5]:
efficientnet_model = EfficientNet(3, 1000)
summary(efficientnet_model, input_size=(1, 3, 224, 224), col_names=['output_size', 'num_params', 'mult_adds'], device='cpu', depth=2)

Layer (type:depth-idx)                                       Output Shape              Param #                   Mult-Adds
EfficientNet                                                 [1, 1000]                 --                        --
├─Sequential: 1-1                                            [1, 32, 112, 112]         --                        --
│    └─Conv2d: 2-1                                           [1, 32, 112, 112]         864                       10,838,016
│    └─BatchNorm2d: 2-2                                      [1, 32, 112, 112]         64                        64
│    └─SiLU: 2-3                                             [1, 32, 112, 112]         --                        --
├─InvertedResidualBlock: 1-2                                 [1, 16, 112, 112]         --                        --
│    └─Sequential: 2-4                                       [1, 16, 112, 112]         1,448                     10,035,848
├─Sequential: 1-3                                

In [6]:
efficientnet_torch_model = models.efficientnet_b0()
summary(efficientnet_torch_model, input_size=(1, 3, 224, 224),
        col_names=['output_size', 'num_params', 'mult_adds'], device='cpu', depth=3)

Layer (type:depth-idx)                                  Output Shape              Param #                   Mult-Adds
EfficientNet                                            [1, 1000]                 --                        --
├─Sequential: 1-1                                       [1, 1280, 7, 7]           --                        --
│    └─Conv2dNormActivation: 2-1                        [1, 32, 112, 112]         --                        --
│    │    └─Conv2d: 3-1                                 [1, 32, 112, 112]         864                       10,838,016
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112]         64                        64
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112]         --                        --
│    └─Sequential: 2-2                                  [1, 16, 112, 112]         --                        --
│    │    └─MBConv: 3-4                                 [1, 16, 112, 112]         1,448          

## Training

In [7]:
from pathlib import Path

TRAIN_RATIO = 0.8
data_dir = Path('./data/')

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

train_ds = datasets.CIFAR100(data_dir, train=True, download=True, transform=transform)
train_ds, val_ds = random_split(train_ds, (TRAIN_RATIO, 1 - TRAIN_RATIO))
val_ds.transform = transform
test_ds = datasets.CIFAR100(data_dir, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
import wandb
from src.engine import *

config = dict(batch_size=64, lr=5e-4, epochs=20, dataset='CIFAR100')
with wandb.init(project='pytorch-study', name='EfficientNet', config=config) as run:
    w_config = run.config
    train_dl = DataLoader(train_ds, batch_size=w_config.batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=w_config.batch_size, shuffle=True)
    
    n_classes = len(train_ds.dataset.classes)
    efficientnet_v2_model = EfficientNet(3, n_classes).to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(efficientnet_v2_model.parameters(), lr=w_config.lr)
    
    loss_history, acc_history = train(efficientnet_v2_model, train_dl, val_dl, criterion, optimizer, w_config.epochs,
                                      DEVICE, run) 

Epoch=20: 100%|██████████| 20/20 [57:16<00:00, 171.84s/it, train_loss=0.159, train_acc=94.81%, val_loss=3.207, val_acc=48.11%]
